In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb

import scipy

sb.set_theme(palette="rocket")

In [ ]:
raw_df = pd.read_csv("PCS_TEST_DETERMINSTIC.csv")
raw_df.info()

## Simple inference
In this section, we will determine the distributions of each column individually.


In [ ]:
# deltas between arrival times
deltas = np.diff(raw_df["Arrival time (sec)"])

sb.histplot(deltas, kde=True)

In [ ]:
# base stations used (should be uniform)
sb.histplot(raw_df["Base station "])


In [ ]:
# call duration

call_dur = raw_df["Call duration (sec)"]

# sb.histplot(call_dur, kde=True)

scipy.stats.t.fit(call_dur)


In [ ]:
# vehicle velocities
sb.histplot(raw_df["velocity (km/h)"], kde=True)
